In [ ]:

!python wiktionary_lookup.py --input "/Users/sanderputs/git/tests/mwe_en_cleaned.xlsx" --sample 200

Wiktionary lookup:  13%|███▎                     | 4/30 [00:05<00:32,  1.25s/it]^C
Traceback (most recent call last):
  File "/Users/sanderputs/git/tests/pymusas_translate/wiktionary_lookup.py", line 217, in <module>
    main()
  File "/Users/sanderputs/git/tests/pymusas_translate/wiktionary_lookup.py", line 198, in main
    url, definition = derive_url_and_definition(phrase)
  File "/Users/sanderputs/git/tests/pymusas_translate/wiktionary_lookup.py", line 151, in derive_url_and_definition
    definition = definition_from_title(guess)
  File "/Users/sanderputs/git/tests/pymusas_translate/wiktionary_lookup.py", line 122, in definition_from_title
    wt = revisions_request(title)
  File "/Users/sanderputs/git/tests/pymusas_translate/wiktionary_lookup.py", line 94, in revisions_request
    res = requests.get(API, params=params, timeout=10, headers={"User-Agent": USER_AGENT}).json()
  File "/Users/sanderputs/git/tests/.venv/lib/python3.13/site-packages/requests/api.py", line 73, in get
   

In [ ]:
!python wiktionary_lookup.py --input "/Users/sanderputs/git/tests/mwe_en_cleaned_wiktionary.csv" --sample 0

Wiktionary lookup:   0%|                        | 1/200 [00:01<03:20,  1.01s/it]^C


In [ ]:
!python wikipedia_lookup.py --input "/Users/sanderputs/git/tests/mwe_en_cleaned_wiktionary.csv" --sample 0

📊 Filtering to 77 rows where wiktionary_sanity_check=False
Wikipedia lookup: 100%|█████████████████████████| 77/77 [02:10<00:00,  1.69s/it]
✅ Results written to /Users/sanderputs/git/tests/mwe_en_cleaned_wiktionary_200_wikipedia_all_filtered.csv
📊 Output contains 77 processed rows and 123 unprocessed rows


In [16]:
import pandas as pd

# Load the output file
df = pd.read_csv("/Users/sanderputs/git/tests/mwe_en_cleaned_wiktionary_200_wikipedia_all_filtered.csv")

# Calculate stats on wikipedia_sanity_check column
print("📊 Wikipedia Sanity Check Statistics")
print("=" * 40)

# Basic value counts
wiki_check_counts = df['wikipedia_sanity_check'].value_counts(dropna=False)
print(f"Value counts:")
for value, count in wiki_check_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {value}: {count} ({percentage:.1f}%)")

print(f"\nTotal rows: {len(df)}")

# Compare with wiktionary_sanity_check
print("\n📊 Cross-tabulation: Wiktionary vs Wikipedia Sanity Checks")
print("=" * 60)
crosstab = pd.crosstab(
    df['wiktionary_sanity_check'], 
    df['wikipedia_sanity_check'], 
    margins=True, 
    dropna=False
)
print(crosstab)

# Success rates
print("\n📊 Success Rates")
print("=" * 20)

# Wiktionary success rate
wikt_success = df['wiktionary_sanity_check'].value_counts().get(True, 0)
wikt_total = df['wiktionary_sanity_check'].notna().sum()
print(f"Wiktionary success rate: {wikt_success}/{wikt_total} ({(wikt_success/wikt_total)*100:.1f}%)")

# Wikipedia success rate (only for processed rows)
wiki_processed = df['wikipedia_sanity_check'].notna()
wiki_success = df[wiki_processed]['wikipedia_sanity_check'].value_counts().get(True, 0)
wiki_total = wiki_processed.sum()
print(f"Wikipedia success rate: {wiki_success}/{wiki_total} ({(wiki_success/wiki_total)*100:.1f}%)")

# Combined success rate (either Wiktionary OR Wikipedia succeeded)
either_success = ((df['wiktionary_sanity_check'] == True) | 
                  (df['wikipedia_sanity_check'] == True)).sum()
print(f"Combined success rate: {either_success}/{len(df)} ({(either_success/len(df))*100:.1f}%)")

# Show some examples of failed lookups
print("\n🔍 Sample of failed lookups (both Wiktionary and Wikipedia failed):")
print("=" * 70)
failed_both = df[
    (df['wiktionary_sanity_check'] == False) & 
    (df['wikipedia_sanity_check'] == False)
]['pos_cleaned'].head(10)

for i, phrase in enumerate(failed_both, 1):
    print(f"{i:2d}. {phrase}")

📊 Wikipedia Sanity Check Statistics
Value counts:
  nan: 123 (61.5%)
  False: 48 (24.0%)
  True: 29 (14.5%)

Total rows: 200

📊 Cross-tabulation: Wiktionary vs Wikipedia Sanity Checks
wikipedia_sanity_check   False  True  NaN  All
wiktionary_sanity_check                       
False                       48    29    0   77
True                         0     0  123  123
All                         48    29    0  200

📊 Success Rates
Wiktionary success rate: 123/200 (61.5%)
Wikipedia success rate: 29/77 (37.7%)
Combined success rate: 152/200 (76.0%)

🔍 Sample of failed lookups (both Wiktionary and Wikipedia failed):
 1. mortgag* * up to the * hilt
 2. Nobel Prize-winner*
 3. recall audition*
 4. special educational difficulties
 5. construction studies
 6. New Town Aston
 7. Antique Worlds
 8. * and * ago
 9. wiring * across
10. drag* * away


In [17]:
!python wordnet_lookup.py --input "/Users/sanderputs/git/tests/mwe_en_cleaned_wiktionary_200_wikipedia_all_filtered.csv" --filter-mode both-false

📊 Filtering to 48 rows using filter mode 'both-false'
WordNet lookup: 100%|███████████████████████████| 48/48 [00:15<00:00,  3.02it/s]
✅ Results written to /Users/sanderputs/git/tests/mwe_en_cleaned_wiktionary_200_wikipedia_all_filtered_wordnet_all_both-false.csv
📊 Processed 48 rows, found 0 WordNet matches (0.0%)
📊 Output contains 48 processed rows and 152 unprocessed rows


In [19]:
# Load the WordNet results
df_wordnet = pd.read_csv("/Users/sanderputs/git/tests/mwe_en_cleaned_wiktionary_200_wikipedia_all_filtered_wordnet_all_both-false.csv")

print("📊 Detailed Analysis of Failed Lookups")
print("=" * 50)

# Create failure categories
df_wordnet['failure_category'] = 'success'

# All three failed
all_failed = (df_wordnet['wiktionary_sanity_check'] == False) & \
             (df_wordnet['wikipedia_sanity_check'] == False) & \
             (df_wordnet['wordnet_sanity_check'] == False)

# Two failed, one succeeded
wikt_only = (df_wordnet['wiktionary_sanity_check'] == True) & \
            (df_wordnet['wikipedia_sanity_check'] == False) & \
            (df_wordnet['wordnet_sanity_check'] == False)

wiki_only = (df_wordnet['wiktionary_sanity_check'] == False) & \
            (df_wordnet['wikipedia_sanity_check'] == True) & \
            (df_wordnet['wordnet_sanity_check'] == False)

wordnet_only = (df_wordnet['wiktionary_sanity_check'] == False) & \
               (df_wordnet['wikipedia_sanity_check'] == False) & \
               (df_wordnet['wordnet_sanity_check'] == True)

# Apply categories
df_wordnet.loc[all_failed, 'failure_category'] = 'all_failed'
df_wordnet.loc[wikt_only, 'failure_category'] = 'wiktionary_only'
df_wordnet.loc[wiki_only, 'failure_category'] = 'wikipedia_only'
df_wordnet.loc[wordnet_only, 'failure_category'] = 'wordnet_only'

# Category counts
print("📊 Failure Categories")
print("-" * 30)
category_counts = df_wordnet['failure_category'].value_counts()
for category, count in category_counts.items():
    percentage = (count / len(df_wordnet)) * 100
    print(f"{category:15}: {count:3d} ({percentage:4.1f}%)")

print(f"\n📊 Complete Failures (all 3 sources failed)")
print("-" * 45)
complete_failures = df_wordnet[all_failed]
print(f"Total complete failures: {len(complete_failures)}")

if len(complete_failures) > 0:
    # Analyze patterns in failed phrases
    failed_phrases = complete_failures['pos_cleaned'].tolist()
    
    # Length analysis
    lengths = [len(phrase.split()) for phrase in failed_phrases]
    print(f"Average word count: {sum(lengths)/len(lengths):.1f}")
    print(f"Length range: {min(lengths)}-{max(lengths)} words")
    
    # Character analysis
    print(f"\n🔍 Pattern Analysis of Failed Phrases:")
    print("-" * 40)
    
    # Wildcards
    with_wildcards = [p for p in failed_phrases if '*' in p]
    print(f"Contains wildcards (*): {len(with_wildcards)}/{len(failed_phrases)} ({len(with_wildcards)/len(failed_phrases)*100:.1f}%)")
    
    # Hyphens
    with_hyphens = [p for p in failed_phrases if '-' in p]
    print(f"Contains hyphens (-): {len(with_hyphens)}/{len(failed_phrases)} ({len(with_hyphens)/len(failed_phrases)*100:.1f}%)")
    
    # Numbers
    with_numbers = [p for p in failed_phrases if any(c.isdigit() for c in p)]
    print(f"Contains numbers: {len(with_numbers)}/{len(failed_phrases)} ({len(with_numbers)/len(failed_phrases)*100:.1f}%)")
    
    # Special characters
    import re
    with_special = [p for p in failed_phrases if re.search(r'[^\w\s\-\*]', p)]
    print(f"Contains special chars: {len(with_special)}/{len(failed_phrases)} ({len(with_special)/len(failed_phrases)*100:.1f}%)")
    
    print(f"\n🎯 Examples of Complete Failures (first 15):")
    print("-" * 45)
    for i, phrase in enumerate(failed_phrases[:15], 1):
        # Show length and patterns
        length = len(phrase.split())
        patterns = []
        if '*' in phrase: patterns.append('*')
        if '-' in phrase: patterns.append('-')
        if any(c.isdigit() for c in phrase): patterns.append('num')
        if re.search(r'[^\w\s\-\*]', phrase): patterns.append('special')
        
        pattern_str = f"[{','.join(patterns)}]" if patterns else ""
        print(f"{i:2d}. {phrase:30} ({length}w) {pattern_str}")

# Show successful rescues by each source
print(f"\n🎯 Successful Rescues by Source:")
print("-" * 35)

if len(df_wordnet[wordnet_only]) > 0:
    print(f"\nWordNet-only successes ({len(df_wordnet[wordnet_only])}):")
    for i, phrase in enumerate(df_wordnet[wordnet_only]['pos_cleaned'].head(10), 1):
        print(f"  {i:2d}. {phrase}")

if len(df_wordnet[wiki_only]) > 0:
    print(f"\nWikipedia-only successes ({len(df_wordnet[wiki_only])}):")
    for i, phrase in enumerate(df_wordnet[wiki_only]['pos_cleaned'].head(10), 1):
        print(f"  {i:2d}. {phrase}")

if len(df_wordnet[wikt_only]) > 0:
    print(f"\nWiktionary-only successes ({len(df_wordnet[wikt_only])}):")
    for i, phrase in enumerate(df_wordnet[wikt_only]['pos_cleaned'].head(10), 1):
        print(f"  {i:2d}. {phrase}")

# Summary table
print(f"\n📊 Success Matrix")
print("-" * 25)
print("Source     | Success | Unique")
print("-" * 25)
wikt_success = (df_wordnet['wiktionary_sanity_check'] == True).sum()
wiki_success = (df_wordnet['wikipedia_sanity_check'] == True).sum()
wordnet_success = (df_wordnet['wordnet_sanity_check'] == True).sum()

print(f"Wiktionary | {wikt_success:7d} | {len(df_wordnet[wikt_only]):6d}")
print(f"Wikipedia  | {wiki_success:7d} | {len(df_wordnet[wiki_only]):6d}")
print(f"WordNet    | {wordnet_success:7d} | {len(df_wordnet[wordnet_only]):6d}")
print("-" * 25)
print(f"All Failed | {len(complete_failures):7d} |")

📊 Detailed Analysis of Failed Lookups
📊 Failure Categories
------------------------------
success        : 152 (76.0%)
all_failed     :  48 (24.0%)

📊 Complete Failures (all 3 sources failed)
---------------------------------------------
Total complete failures: 48
Average word count: 3.3
Length range: 2-8 words

🔍 Pattern Analysis of Failed Phrases:
----------------------------------------
Contains wildcards (*): 24/48 (50.0%)
Contains hyphens (-): 1/48 (2.1%)
Contains numbers: 1/48 (2.1%)
Contains special chars: 0/48 (0.0%)

🎯 Examples of Complete Failures (first 15):
---------------------------------------------
 1. mortgag* * up to the * hilt    (7w) [*]
 2. Nobel Prize-winner*            (2w) [*,-]
 3. recall audition*               (2w) [*]
 4. special educational difficulties (3w) 
 5. construction studies           (2w) 
 6. New Town Aston                 (3w) 
 7. Antique Worlds                 (2w) 
 8. * and * ago                    (4w) [*]
 9. wiring * across              